# DL2 (WIP)

## 학습 내용

### Convolutional Neural Networks

이미지 등의 grid-like 데이터를 convolutional kernel (또는 filter)을 이용하여 지엽적인 정보를 담은 feature map들을 생성한 뒤 infer하는 네트워크이다.

#### Convolution
$$ s(t) = \int x(a)w(t-a)da = (x \ast w)(t) $$
Note that it is *comminitative*.

#### Motivations
- *Sparce connectivity*: 한 input unit이 적은 수의 sparce한 output unit들만과 상호작용하므로 parameter의 수를 효율적으로 줄일 수 있다.
- *Parameter sharing*: 같은 parameter를 가진 kernel을 여러 input에 재사용함으로서 모델의 크기플 줄인다.
- *Equivarience*: 일정하게 바뀌는 input에 대해 비슷한 output을 만든다. (e.g. shift된 이미지 input에 대한 edge detection)

#### Padding
- `"valid"`: without padding
- `"same"`: zero padding

#### Pooling layers
To *subsample*.
Max pooling을 많이 사용한다.
이는 작고 중요하지 않은 input의 차이를 무시하도록 *(invariant하도록)* 한다.

#### Architecture
Convolution layers, Pooling layers 를 쌓은 후 Fully connected layer들을 쌓아 Back-propagation 과정에서 각 layer들을 최적화시킨다. Output layer에 가까운 filter일수록 더 복잡한 feature를 구분할 수 있다.

### Notable CNN architctures

#### LeNet-5
1998, Yann LeCun

| Layer | Type | Size x Maps | Activation |
| - | -| - | - |
| Out | Fully connected | 10 | RBF |
| F6 | Fully connected | 84 | tanh |
| C5 | Convolution | 1x1x120 | tanh |
| A4 | Avg pooling | 5x5x16 | tanh |
| C3 | Convolution | 10x10x16 | tanh |
| A2 | Avg pooling | 14x14x6 | tanh |
| C1 | Convolution | 28x28x6 | tanh |
| In | Input | 32x32x1 | - |

- 28x28 MNIST 데이터들을 위해 제작됨 (with zero-padding).
- Output layer에서는 input vector와 output vector의 Euclidian distance의 square을 출력.

### AlexNet
2012, Alex Krizhevsky et al.

| Layer | Type | Size x Maps | Activation |
| - | -| - | - |
| Out | Fully connected | 1000 | Softmax |
| F9 | Fully connected | 4096 | ReLU |
| F8 | Fully connected | 4096 | ReLU |
| C7 | Convolution | 13x13x256 | ReLU |
| C6 | Convolution | 13x13x384 | ReLU |
| C5 | Convolution | 13x13x384 | ReLU |
| S4 | Max pooling | 13x13x256 | - |
| C3 | Convolution | 27x27x256 | ReLU |
| S2 | Max pooling | 27x27x96 | - |
| C1 | Convolution | 55x55x96 | ReLU |
| In | Input | 227x227x3 | - |

- ImageNet을 위해 제작됨.
- Training 과정에서:
  - 50% Dropout이 사용됨 (`F8`, `F9` layer)
  - Data augmentation이 사용됨 (ramdomly shifting the training images, flipping, changing the light)
- *Competitive normalizaation step* right after the ReLU step of layers C1 and C3, called *local response normalization* (LRN)
  - The most strongly activated neurons inhibit other neurons located at the same position in neightboring feature maps

### GoogLeNet
2014, Christian Szegedy et al.

- Inception module의 사용
  - 파라미터의 수를 줄이면서 효율적인 연산.
  - 1x1 Convolution layer (w/ stride=1)의 사용
    - Input보다 적은 feature map을 출력하여 dimensionality를 줄임 (bottlenack layers)
  - 각 inception module의 output에는 depth concat 사용.
- 여러 inception model들과 LRN, max pool 등을 쌓다가 Global avg pool을 쌓는다. 이는 spatatial information을 drop한다 (목적이 localization이 아닌 classification이기 때문에 상관이 없다). 
- 이 모델은 dimentsional reduction을 강하게 적용했으므로 전 모델들과는 달리 fully connected layer는 하나만 사용한다. 덕분에 parameter의 수를 줄일 수 있다.

### ResNet
2015, Kaming He et al.

- Residual Network *("ResNet")* 의 사용
  - Skip connections (or, "shortcut connections")의 사용: 타겟 function $ h(\boldsymbol{x}) $ 에 input $ \boldsymbol{x} $ 를 넣어 $ h(\boldsymbol{x}) = f(\boldsymbol{x}) + \boldsymbol{x} $ 라고 한다면 network는 $ h(\boldsymbol{x}) $ 가 아닌 $ f(\boldsymbol{x}) = h(\boldsymbol{x}) - \boldsymbol{x} $ 를 배우게 됨.
  - 초기엔 모든 weight가 $ \boldsymbol{0} $ 이기 때문에 $ h $ 는 identity function으로 시작하는데, 이때 원하는 $ h $ 가 identity function과 비슷할 때 효율적인 결과를 보인다.

### Training techniques

#### Object localization
metric은 *Intersection over Union* (IoU)를 사용한다.
  - (Label과 Prediction의 intersection) / (Union)

#### Object detection
- 가장 간단한 방법은 이미지를 grid로 나누고 여러 grid를 포함하는 bounding box를 옮겨가며 계산하는 것이지만, 같은 영역을 여러번 계산한다는 비효율성이 있다.
- Non-max suppression은 이 문제를 해결한다. 꽃을 detect하는 모델을 예로 들자면
  1. 이미지에 "꽃이 있는가"를 추측하는 *objectness* 출력을 CNN에 추가한다. 그 후 objectness score가 낮은 bounding box를 제외해 불필요한 검사를 줄인다.
  2. 높은 objectness score를 가진 bounding box와 겹치는 다른 bounding box를 제외하여 여러번 검사하는 경우를 줄인다.
- *You Only Look Once* (YOLO): 말 그대로 한번만 보는 방식이다. 특징으로는:
  - 한 grid 당 하나가 아닌 5개의 bounding box를 만드며, 각 bounding box는 objectness score를 가지고 있다.
  - 절대적인 좌표 대신 각 grid에 대한 상대적인 offset을 구한다.
  - Training 전 bounding box의 dimension (w: h ratio)인 *anchor box*를 구한다. 

#### Semantic segmentation
- Input image의 크기가 큼을 요구할 때가 많은데, 이때 upsampling 과정에서 *transposed convolution layer*를 사용한다.
  - 이미지 사이를 empty rows and columns로 채워 그 위에서 convolution을 수행한다.

## More...
- p324. Toeplitz matrix & Doubly block circulant matrix
- p334 - 349